In [1]:
import torch
from tqdm import tqdm, trange


# Importing specific libraries for data prerpcessing, model archtecture choice, training and evaluation
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
!pip install torchvision 
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
!python -m pip install -U gensim
!python -m pip install -U keras
!python -m pip install tensorflow
from sklearn import datasets
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.utils.multiclass import unique_labels
import string
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn import model_selection
import pickle
import pandas as pd
import string
import re
from keras.preprocessing.text import Tokenizer

Requirement already up-to-date: gensim in c:\users\adria\anaconda3\lib\site-packages (4.0.1)
Requirement already up-to-date: keras in c:\users\adria\anaconda3\lib\site-packages (2.4.3)


In [2]:
#Import data
#Remove punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
TestSet = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Dataset_Aicrowd/test_nolabel.csv"
TestSet = pd.read_csv(TestSet)
TestSet.columns = ['sentence']

Cognates = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/Cognates.csv"
Cognates = pd.read_csv(Cognates)

CognatesGlossary = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates/glossaryCognates.csv"
CognatesGlossary = pd.read_csv(CognatesGlossary)

TestSet["sentence"] = TestSet['sentence'].apply(remove_punctuations)
TestSet.head(-1)

,sentence
0,Il est devenu courant de déplorer que la Franc...
1,Sur les flancs et les derrières de la noce tou...
2,Jaime aussi beaucoup sa voix une voix un peu g...
3,En partenariat avec lINPES Institut National d...
4,Il se réveilla seulement quand il entendit un ...
...,...
1194,On ne sattaque pas impunément à un repère cult...
1195,Autrefois vous viviez retirée vous fuyiez une...
1196,On a un prof super cool genre métissé cheveux ...
1197,Il nengendre aucun mouvement alternatif ce qui...


In [3]:
#Remove cognates from the test data

TestSet2 = TestSet
[(lambda x: re.sub(r'|'.join(Cognates), '', x))(x) for x in TestSet2]
TestSet2.head(-1)

,sentence
0,Il est devenu courant de déplorer que la Franc...
1,Sur les flancs et les derrières de la noce tou...
2,Jaime aussi beaucoup sa voix une voix un peu g...
3,En partenariat avec lINPES Institut National d...
4,Il se réveilla seulement quand il entendit un ...
...,...
1194,On ne sattaque pas impunément à un repère cult...
1195,Autrefois vous viviez retirée vous fuyiez une...
1196,On a un prof super cool genre métissé cheveux ...
1197,Il nengendre aucun mouvement alternatif ce qui...


In [4]:
#Load the model that was saved locally on disk
filename = 'withoutcognates_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
model = loaded_model

In [5]:
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [6]:
text = TestSet2['sentence'].to_list()
comments = text
epochs = 5
MAX_LEN = 128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True,max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [8]:
# Apply the finetuned model (Camembert)
flat_pred = []
nb_classes = 6
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  model(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())

In [12]:
#Create an empty list and iterate over the prediction to get all the results
Prediction = []
for i in range(len(flat_pred)):
    print('Comment: ', comments[i])
    print('Label', flat_pred[i])
    Prediction.append(flat_pred[i])

Comment:  Il est devenu courant de déplorer que la France ait acquis la place de championne dEurope des prélèvements obligatoires expression qui englobe les impôts dÉtat les impôts locaux les taxes et les cotisations sociales  la part du Produit intérieur brut PIB que les autorités prélèvent sur la richesse nationale a atteint 48 4  alors que la moyenne dEurope est à 40 2 
Label 4
Comment:  Sur les flancs et les derrières de la noce tout le village en rumeur bourdonnait comme un essaim
Label 5
Comment:  Jaime aussi beaucoup sa voix une voix un peu grave quon nattend pas chez une fille aussi frêle
Label 1
Comment:  En partenariat avec lINPES Institut National de Prévention et dÉducation pour la Santé elle a mis en place un programme sur laddiction aux écrans qui aborde le fonctionnement du cerveau à la limite des neurosciences
Label 4
Comment:  Il se réveilla seulement quand il entendit un bruit étrange et quand il ouvrit les yeux il vit que tous les serpents étaient réveillés
Label 2
C

Comment:  Aujourdhui elle contraint les spécialistes de léducation à une réflexion aussi douloureuse quembarrassante
Label 3
Comment:  Le bulletin de notes du gouvernement a piètre allure
Label 3
Comment:  Oui je lai reçue hier
Label 1
Comment:  On lui accorda pour compagnes et afin de laider lors de son accouchement deux femmes des plus sages furent cloîtrées avec elle
Label 3
Comment:  Oui avec ce grand soleil je commence à réfléchir à des projets de vacances
Label 1
Comment:  Nous neûmes aucun mal à retrouver lhomme parmi la foule des voyageurs qui convergeaient vers la sortie
Label 3
Comment:  Comme pour les autres sciences la démarche des enseignants intéressés par les ressources pédagogiques et les formations de La Main à la pâte est une démarche volontaire
Label 5
Comment:  La plateforme française du commerce équitable PFCE réseau des treize principales associations et entreprises françaises a élaboré sa propre charte
Label 4
Comment:  Je voudrais réserver une chambre double pou

Comment:  Lundi je retourne à lécole
Label 0
Comment:  Il doit se dépêcher pour ne pas être en retard à lécole
Label 1
Comment:  Enfin les producteurs sinquiètent de la pollution que peuvent entraîner ces équipements qui contiennent parfois des substances dangereuses
Label 3
Comment:  Un lieu plutôt réservé aux repas en famille ou entre amis
Label 2
Comment:  Il veut tout manger
Label 0
Comment:  Je suis battu
Label 0
Comment:  Je suis désolée mais je ne suis pas libre ce weekend
Label 0
Comment:  Lasile est le creuset terrible où se forge lidentité du vagabond
Label 5
Comment:  Deux messieurs dun certain âge se promènent le long dune rivière
Label 2
Comment:  Une étude montre que ce vaccin offre un taux de protection élevé qui permet déviter lhospitalisation
Label 3
Comment:  Et on ira manger une gaufre juste après ça te dit 
Label 0
Comment:  Un peuple dont on ne connaît pas le nom
Label 1
Comment:  Jai bien reçu ton message
Label 1
Comment:  Ne renonce jamais aux cruautés qui vivifi

Label 1
Comment:  répétait Fabrice il est donc criminel de vouloir défendre la patrie 
Label 3
Comment:  Votre place aujourdhui là voyons entre nous vous a coûté combien 
Label 2
Comment:  Cest pourquoi dans le désert les premiers déplacements seffectuent tôt le matin avant 8 h 30 pour profiter de la fraîcheur persistante de la nuit
Label 2
Comment:  Il savait que le souvenir même du piano faussait encore le plan dans lequel il voyait les choses de la musique que le champ ouvert au musicien nest pas un clavier mesquin de sept notes mais un clavier incommensurable encore presque tout entier inconnu où seulement çà et là séparées par dépaisses ténèbres inexplorées quelquesunes des millions de touches de tendresse de passion de courage de sérénité qui le composent chacune aussi différente des autres quun univers dun autre univers ont été découvertes par quelques grands artistes qui nous rendent le service en éveillant en nous le correspondant du thème quils ont trouvé de nous montrer quel

Label 5
Comment:  Le terme de géohistoire défini par Fernand Braudel lorsquil choisit la Méditerranée comme objet de sa thèse en 1949 a été repris dans les années 1980 par des géographes qui ont utilisé leurs outils récents de pensée des territoires pour réexaminer non seulement les espaces historiquement révolus mais simultanément considérer les régions du temps les périodes comme des objets formalisables
Label 5
Comment:  Cest le château
Label 0
Comment:  Jaime le fromage
Label 0
Comment:  Non dit lun de ces messieurs nous vous demandons si vous naimez pas tendrement le roi des Bulgares
Label 2
Comment:  Les membres sont au nombre de quatre  deux en haut et deux en bas
Label 1
Comment:  Il faut du temps et du courage pour soigner toute cette ménagerie En plus des animaux il faut aussi soccuper du potager de mon papy
Label 2
Comment:  Vous habitez loin dici 
Label 0
Comment:  Ce que la porte était bien incapable de faire
Label 2
Comment:  Cette année nous avons demandé à nos parents d

Comment:  Madame la baronne qui pesait environ trois cent cinquante livres sattirait par là une très grande considération et faisait les honneurs de la maison avec une dignité qui la rendait encore plus respectable
Label 3
Comment:  Alors se modifièrent rapidement les règles sévères quelle mavait prescrites
Label 4
Comment:  Questce que cest 
Label 0
Comment:  Leurs capacités de déduction et de connexion sont certes développées leur force mécanique est puissante leur précision micrométrique mais les robots restent des machines au service de lhomme
Label 4
Comment:  Quel est cet animal 
Label 0
Comment:  Je técris pour te dire que je vais partir cet été en Corse avec ma famille
Label 1
Comment:  Ashton lui lançait en notes graves des provocations homicides Lucie poussait sa plainte aiguë Arthur modulait à lécart des sons moyens et la bassetaille du ministre ronflait comme un orgue tandis que les voix de femmes répétant ses paroles reprenaient en choeur délicieusement
Label 5
Comment:  J

Comment:  Aujourdhui la classe inversée une innovation pédagogique qui rencontre un succès sans précédent
Label 3
Comment:  OK pas de problème
Label 0
Comment:  Avant je me disais que ma vie était une tragédie
Label 1
Comment:  Vous préparez le repas 
Label 0
Comment:  Avec les prévisions météo on peut savoir quel temps il fera le soir le lendemain ou dans la semaine
Label 1
Comment:  Sortons faisons quelquechose nimporte quoi juste pour bouger un peu
Label 1
Comment:  Le curé possédait également un âne solide obstiné juste ce quil fallait très doux qui lui faisait toutes ses besognes
Label 3
Comment:  Les activités ne manquent pas 
Label 1
Comment:  Linfluence du microbiote sur notre comportement a pu être observée
Label 3
Comment:  Je vous appellerais au cas où il y a aurait un problème
Label 1
Comment:  Anna alla dans la cuisine où elle se sentait chez elle
Label 1
Comment:  Plusieurs années avant de se faire connaitre sous le nom de Saul Goodman James McGill était un arnaqueur à Ci

In [13]:
#Remove and replace the number labels into the correct ones
Prediction
Prediction = ['A1' if i==0 else i for i in Prediction]
Prediction = ['A2' if i==1 else i for i in Prediction]
Prediction = ['B1' if i==2 else i for i in Prediction]
Prediction = ['B2' if i==3 else i for i in Prediction]
Prediction = ['C1' if i==4 else i for i in Prediction]
Prediction = ['C2' if i==5 else i for i in Prediction]
Prediction
df = pd.DataFrame(Prediction, columns = ["difficulty"])


In [14]:
df

,difficulty
0,C1
1,C2
2,A2
3,C1
4,B1
...,...
1195,C2
1196,B2
1197,B2
1198,C2


In [16]:
#Import the prediction into a CSV file to try our model
df.to_csv('prediction3withNoCognates.csv', encoding='utf-8-sig', index=False)